<a href="https://colab.research.google.com/github/gnodnooh/hydroclimatology/blob/main/ABCD_sim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Load data
from google.colab import drive
drive.mount('/content/drive')
prec_fcst = np.loadtxt('/content/drive/MyDrive/lab_session/Precip_fcst_JAS.txt')  # Precipitation forecast data
prec_clim = np.loadtxt('/content/drive/MyDrive/lab_session/Koka_Precip_climatology.txt')  # Precipitation climatology data
evap_clim = np.loadtxt('/content/drive/MyDrive/lab_session/Koka_evap.txt')  # Evaporation climatology
obs = np.loadtxt('/content/drive/MyDrive/lab_session/Koka_SF_obs_81_00.txt')  # Observed data

print('Precipitation forecast data shape:', prec_fcst.shape)
print('Precipitation climatology data shape:', prec_clim.shape)
print('Evaporation climatology shape:', evap_clim.shape)
print('Observed data shape:', obs.shape)

In [ ]:
# Combine precipitation forecast and climatology
prec_mo_yr = np.tile(prec_clim[:,None], (1, 20))  # Make 20 years of monthly climatology precipitation
prec_mo_yr[6:9, :] = prec_fcst  # Substitute in forecast values for Jul-Sept (indices 7-9 in MATLAB)
prec = prec_mo_yr.T.flatten()
prec.shape

In [ ]:
# Make climatological evaporation for 20 years
evap = np.tile(evap_clim, (1, 20)).flatten()
evap.shape

In [ ]:
# Parameters
a = 0.9
b = 285
c = -5
d = 0
# For IGUATU, use parameters below:
# a = 0.984
# b = 284.009
# c = 0.488
# d = 0

# Define number of periods (months) plus 1 for initialization period
NPER = len(prec)

# Define variables. First entry is for initialization period
W = np.zeros(NPER)
S = np.zeros(NPER)
Y = np.zeros(NPER)
E = np.zeros(NPER)  # E is not used in the original MATLAB code
G = np.zeros(NPER)
Qmod = np.zeros(NPER)

# Loop for soil moisture accounting and surface runoff
for i in range(NPER - 1):
    W[i + 1] = prec[i + 1] + S[i]
    Y[i + 1] = ((W[i + 1] + b) / (2 * a)) - np.sqrt(((W[i + 1] + b) / (2 * a))**2 - (W[i + 1] * b) / a)
    S[i + 1] = Y[i + 1] * np.exp(-evap[i + 1] / b)
    G[i + 1] = (c * (W[i + 1] - Y[i + 1]) + G[i]) / (1 + d)
    Qmod[i + 1] = (1 - c) * (W[i + 1] - Y[i + 1]) + d * G[i + 1]

# Plot observed vs modeled
fig = plt.figure(figsize=(10, 6))
plt.plot(range(240), obs, 'k', label='obs')
plt.plot(range(240), Qmod, 'b', label='model')
plt.legend()
plt.xlabel('Months')
plt.ylabel('Streamflow')
plt.title('Observed vs Modeled Streamflow')
plt.show()

# Calculate and print the correlation coefficient
corr_coef = np.corrcoef(obs, Qmod)
print(f'Correlation Coefficient:\n{corr_coef}')